<a href="https://colab.research.google.com/github/azadkhah/speech2text/blob/main/ASR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Fri Jun 10 07:15:34 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   63C    P8    11W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os
os.chdir('/content/drive/MyDrive/speech2text')

In [4]:
%env LC_ALL=C.UTF-8
%env LANG=C.UTF-8
%env TRANSFORMERS_CACHE=/content/cache
%env HF_DATASETS_CACHE=/content/cache
%env CUDA_LAUNCH_BLOCKING=1

env: LC_ALL=C.UTF-8
env: LANG=C.UTF-8
env: TRANSFORMERS_CACHE=/content/cache
env: HF_DATASETS_CACHE=/content/cache
env: CUDA_LAUNCH_BLOCKING=1


In [5]:
!ls

cv-corpus-9.0-2022-04-27  cv-corpus-9.0-2022-04-27-fa.tar.gz


In [ ]:
%%capture
!pip install -U datasets
!pip install -U transformers
!pip install torchaudio
!pip install librosa
!pip install jiwer
!pip install hazm
!pip install num2fawords

In [ ]:
%pip install pydub
%pip install pythainlp==2.3.1
%pip install ipywidgets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 11.0 MB 5.5 MB/s 
     |████████████████████████████████| 965 kB 30.8 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import os
import re
from typing import List, Dict, Tuple

import pandas as pd
from scipy.io import wavfile
from pythainlp.tokenize import word_tokenize


# ipython
import IPython.display as ipd
import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np
from tqdm.auto import tqdm

In [ ]:
!tar -xzf cv-corpus-9.0-2022-04-27-fa.tar.gz

In [ ]:
!rm -rf fa.tar.gz

In [ ]:
from datasets import load_dataset, load_metric

import pandas as pd
import numpy as np

import hazm
from num2fawords import words, ordinal_words
from tqdm import tqdm

from sklearn.model_selection import train_test_split

import os
import string
import six
import re
import glob

In [ ]:
_normalizer = hazm.Normalizer()

chars_to_ignore = [
    ",", "?", ".", "!", "-", ";", ":", '""', "%", "'", '"', "�",
    "#", "!", "؟", "?", "«", "»", "،", "(", ")", "؛", "'ٔ", "٬",'ٔ', ",", "?", 
    ".", "!", "-", ";", ":",'"',"“", "%", "‘", "”", "�", "–", "…", "_", "”", '“', '„',
    'ā', 'š',
#     "ء",
]

# In case of farsi
chars_to_ignore = chars_to_ignore + list(string.ascii_lowercase + string.digits)

chars_to_mapping = {
    'ك': 'ک', 'دِ': 'د', 'بِ': 'ب', 'زِ': 'ز', 'ذِ': 'ذ', 'شِ': 'ش', 'سِ': 'س', 'ى': 'ی',
    'ي': 'ی', 'أ': 'ا', 'ؤ': 'و', "ے": "ی", "ۀ": "ه", "ﭘ": "پ", "ﮐ": "ک", "ﯽ": "ی",
    "ﺎ": "ا", "ﺑ": "ب", "ﺘ": "ت", "ﺧ": "خ", "ﺩ": "د", "ﺱ": "س", "ﻀ": "ض", "ﻌ": "ع",
    "ﻟ": "ل", "ﻡ": "م", "ﻢ": "م", "ﻪ": "ه", "ﻮ": "و", 'ﺍ': "ا", 'ة': "ه",
    'ﯾ': "ی", 'ﯿ': "ی", 'ﺒ': "ب", 'ﺖ': "ت", 'ﺪ': "د", 'ﺮ': "ر", 'ﺴ': "س", 'ﺷ': "ش",
    'ﺸ': "ش", 'ﻋ': "ع", 'ﻤ': "م", 'ﻥ': "ن", 'ﻧ': "ن", 'ﻭ': "و", 'ﺭ': "ر", "ﮔ": "گ",
        
    # "ها": "  ها", "ئ": "ی",
    "۱۴ام": "۱۴ ام",
        
    "a": " ای ", "b": " بی ", "c": " سی ", "d": " دی ", "e": " ایی ", "f": " اف ",
    "g": " جی ", "h": " اچ ", "i": " آی ", "j": " جی ", "k": " کی ", "l": " ال ",
    "m": " ام ", "n": " ان ", "o": " او ", "p": " پی ", "q": " کیو ", "r": " آر ",
    "s": " اس ", "t": " تی ", "u": " یو ", "v": " وی ", "w": " دبلیو ", "x": " اکس ",
    "y": " وای ", "z": " زد ",
    "\u200c": " ", "\u200d": " ", "\u200e": " ", "\u200f": " ", "\ufeff": " ",
}


def multiple_replace(text, chars_to_mapping):
    pattern = "|".join(map(re.escape, chars_to_mapping.keys()))
    return re.sub(pattern, lambda m: chars_to_mapping[m.group()], str(text))

def remove_special_characters(text, chars_to_ignore_regex):
    text = re.sub(chars_to_ignore_regex, '', text).lower() + " "
    return text

def normalizer(text, chars_to_ignore=chars_to_ignore, chars_to_mapping=chars_to_mapping):
    chars_to_ignore_regex = f"""[{"".join(chars_to_ignore)}]"""
    text = text.lower().strip()

    text = _normalizer.normalize(text)
    text = multiple_replace(text, chars_to_mapping)
    text = remove_special_characters(text, chars_to_ignore_regex)
    text = re.sub(" +", " ", text)
    _text = []
    for word in text.split():
        try:
            word = int(word)
            _text.append(words(word))
        except:
            _text.append(word)
            
    text = " ".join(_text) + " "
    text = text.strip()

    if not len(text) > 0:
        return None
    
    return text + " "

In [ ]:
abs_path_to_data = os.path.join("/content/drive/MyDrive/speech2text/cv-corpus-9.0-2022-04-27/fa")
!ls {abs_path_to_data}

clips	 invalidated.tsv  reported.tsv	train.tsv
dev.tsv  other.tsv	  test.tsv	validated.tsv


In [ ]:
cv_root: str = "./cv-corpus-9.0-2022-04-27"

train: pd.DataFrame = pd.read_csv(f"{cv_root}/fa/train.tsv", delimiter="\t")
train["set"] = "train"
dev: pd.DataFrame = pd.read_csv(f"{cv_root}/fa/dev.tsv", delimiter="\t")
dev["set"] = "dev"
test: pd.DataFrame = pd.read_csv(f"{cv_root}/fa/test.tsv", delimiter="\t")
test["set"] = "test"
data: pd.DataFrame = pd.read_csv(f"{cv_root}/fa/validated.tsv", delimiter="\t")

command: str = "sox {mp3_path} -t wav -r {sr} -c 1 -b 16 - |"

#     break
    
df = pd.DataFrame(train)
df.head()


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,client_id,path,sentence,up_votes,down_votes,age,gender,accents,locale,segment,set
0,dfdeffdc55c8a41487b142ba5afb354d8eb081822ed326...,common_voice_fa_25244256.mp3,او نسبت به هم مدرسهای خود محبت کودکانه داشت.,2,0,NaN,NaN,NaN,fa,NaN,train
1,dfdeffdc55c8a41487b142ba5afb354d8eb081822ed326...,common_voice_fa_25244257.mp3,او هرگونه اطلاع دربارهی توطئه را انکار کرد.,2,0,NaN,NaN,NaN,fa,NaN,train
2,dfdeffdc55c8a41487b142ba5afb354d8eb081822ed326...,common_voice_fa_25244258.mp3,اکنون میتوانی همهی نقشههای خود را آشکار کنی.,2,0,NaN,NaN,NaN,fa,NaN,train
3,dfdeffdc55c8a41487b142ba5afb354d8eb081822ed326...,common_voice_fa_25244259.mp3,اکثر پرندگان تخم میگذارند.,2,0,NaN,NaN,NaN,fa,NaN,train
4,dfdeffdc55c8a41487b142ba5afb354d8eb081822ed326...,common_voice_fa_25244260.mp3,با این تهدیدها نمیتوانی مرا بترسانی!,2,0,NaN,NaN,NaN,fa,NaN,train


In [ ]:
print(f"Step 0: {len(df)}")

df["status"] = df["path"].apply(lambda path: True if os.path.exists(path) else None)
df = df.dropna(subset=["path"])
df = df.drop("status", 1)
print(f"Step 1: {len(df)}")

# df["prev_sentence"] = df["sentence"]
df["sentence"] = df["sentence"].apply(lambda t: normalizer(t))
df = df.dropna(subset=["sentence"])
print(f"Step 2: {len(df)}")

# df = df[["prev_sentence", "sentence", "path"]]
# df = df[["sentence", "path"]]
df = df.sample(frac=1)
df = df.reset_index(drop=True)
df.head()


Step 0: 24150


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  """


Step 1: 24150
Step 2: 24150


,client_id,path,sentence,up_votes,down_votes,age,gender,accents,locale,segment,set
0,a6e84b60065be5e12b28ae48d1df461a35de7ceb6f058f...,common_voice_fa_28448678.mp3,آنان خواستار شمارش مجدد آرا شدند,2,0,thirties,male,NaN,fa,NaN,train
1,a6e84b60065be5e12b28ae48d1df461a35de7ceb6f058f...,common_voice_fa_28424301.mp3,ساختمان کارخانهی جدید را زمانبندی کردند,2,0,NaN,NaN,NaN,fa,NaN,train
2,d9e326521966c7d01f1a1011e73acbf0f194800788904e...,common_voice_fa_22774810.mp3,ابوبکر,2,0,twenties,male,NaN,fa,NaN,train
3,57d6c1695947f141cd774bb8c8435e7e20bc7b2a83cd78...,common_voice_fa_18226342.mp3,فردا بیا تا ازت یه تست بگیرم,2,0,teens,male,NaN,fa,NaN,train
4,02891c7f20efefaece0df2cce7f74e0fedb8bd47d2930a...,common_voice_fa_22772083.mp3,آیگین,2,0,twenties,male,NaN,fa,NaN,train


In [ ]:
main_vocab = ["ح", "چ", "ج", "ث", "ت", "پ", "ب", "آ", "ا", "ش", "س", "ژ", "ز", "ر", "ذ", "د", "خ", "ق", "ف", "غ", "ع", "ظ", "ط", "ض", "ص", "ی", "ه", "و", "ن", "م", "ل", "گ", "ک"]
text = " ".join(df["sentence"].values.tolist())
vocab = list(sorted(set(text)))

for v in main_vocab:
    if v not in vocab:
        print("v", v)

print(len(main_vocab), len(vocab))
print(vocab)

33 37
[' ', '=', 'ء', 'آ', 'ئ', 'ا', 'ب', 'ت', 'ث', 'ج', 'ح', 'خ', 'د', 'ذ', 'ر', 'ز', 'س', 'ش', 'ص', 'ض', 'ط', 'ظ', 'ع', 'غ', 'ف', 'ق', 'ل', 'م', 'ن', 'ه', 'و', 'پ', 'چ', 'ژ', 'ک', 'گ', 'ی']


In [ ]:
import torchaudio
import librosa
import IPython.display as ipd
import numpy as np

idx = np.random.randint(0, len(df))
print(idx)
sample = df.iloc[idx]

path = sample["path"]
print(sample["sentence"], "\n")
speech, sr = torchaudio.load("/content/drive/MyDrive/speech2text/cv-corpus-9.0-2022-04-27/fa/clips/"+path)
speech = speech[0].numpy().squeeze()

speech = librosa.resample(np.asarray(speech), sr, 16_000)
ipd.Audio(data=np.asarray(speech), autoplay=True, rate=16000)

8100
او در حل اختلاف میانجیگری کرد  



In [ ]:
train_df, test_df = train_test_split(df, test_size=0.1, random_state=101, stratify=df["gender"])
train_df=pd.DataFrame(train)
test_df
train_df = train_df[["path", "sentence"]]
train_df = train_df.reset_index(drop=True)

test_df = test_df[["path", "sentence"]]
test_df = test_df.reset_index(drop=True)

print(train_df.shape)
print(test_df.shape)